In [ ]:
t1 = [(0,1), (1,2), (2,3), (3,4), (4,5), (5,6), (6,7), (7,8), (8,9), (9,0)]
t2 = [(0,1), (1,9), (9,2), (2,8), (8,7), (7,6), (6,5), (5,4), (4,3), (3,2)]

In [ ]:
def reverseTuple(lstOfTuple):
    return [tup[::-1] for tup in lstOfTuple]

In [ ]:
reverse_t2 = reverseTuple(t2)
print(reverse_t2)

In [ ]:
new_cycle = []
for i in t1:
    if i in t2 or i in reverse_t2:
        new_cycle.append(i)

In [ ]:
new_cycle

In [ ]:
left_t1 = [i for i, j in t1]
right_t1 = [j for i, j in t1]
left_t2 = [i for i, j in t2]
right_t2 = [j for i, j in t2]

In [ ]:
left_t1

In [ ]:
right_t1

In [ ]:
left_t2

In [ ]:
right_t2

In [ ]:
left_new_cycle = [i for i, j in new_cycle]
right_new_cycle = [j for i, j in new_cycle]
    

In [ ]:
print("Brakuje krawędzi, która kończy się w wierzchołkach:")
for i in left_new_cycle:
    if i not in right_new_cycle:
        print(i)

In [ ]:
print("Brakuje krawędzi, która zaczyna się w wierzchołkach:")
for i in right_new_cycle:
    if i not in left_new_cycle:
        print(i)

In [ ]:
new_cycle_vertices = list(set(left_new_cycle+right_new_cycle))
print(left_new_cycle)
print(right_new_cycle)
t1_vertices = list(set(left_t1))

print(t1_vertices)
print("Wierzchołki, w których ani się nie zaczyna ani się nie kończy żadna krawędź:")
for i in t1_vertices:
    if i not in new_cycle_vertices:
        print(i)

In [ ]:
import pandas as pd
import numpy as np
import random as rd
import matplotlib.pyplot as plt
import seaborn as sns
import time
from tqdm import tqdm

In [ ]:
#funkcja obliczająca odległość między dwoma wierzchołkami z matematycznym zaokrągleniem
def distance(x1,x2,y1,y2):
    return np.floor(np.sqrt((x1-x2)**2+(y1-y2)**2)+0.5)

In [ ]:
#funkcja obliczająca całkowitą długość cyklu
def distance_sum_old(cycle,matrix):
    distance_list = [matrix[cycle[i]][cycle[0]] if i==len(cycle)-1 else matrix[cycle[i]][cycle[i+1]]
                    for i in range(len(cycle))]
    
    return sum(distance_list)

In [ ]:
#funkcja obliczająca całkowitą długość cyklu
def distance_sum(cycle,matrix):
    result = 0
    for pair in cycle:
        result+= matrix[int(pair[0])][int(pair[1])]
    return result

In [ ]:
#wczytanie pierwszej instancji (kroA200)

instance = pd.read_csv('kroA200.tsp', sep=' ', names=['node', 'x', 'y'], skiprows=6, skipfooter=1,
                      engine='python')
kroA200 = []

for index1, row1 in instance.iterrows():
    tmp = []
    for index2, row2 in instance.iterrows():
        if (index1==index2):
            tmp.append(0)
        else:
            tmp.append(distance(row1['x'],row2['x'],row1['y'],row2['y']))
    kroA200.append(tmp)
    
np.array(kroA200)

In [ ]:
#wczytanie drugiej instancji (kroB200)

instance2 = pd.read_csv('kroB200.tsp', sep=' ', names=['node', 'x', 'y'], skiprows=6, skipfooter=1,
                      engine='python')

kroB200 = []

for index1, row1 in instance2.iterrows():
    tmp = []
    for index2, row2 in instance2.iterrows():
        if (index1==index2):
            tmp.append(0)
        else:
            tmp.append(distance(row1['x'],row2['x'],row1['y'],row2['y']))
    kroB200.append(tmp)

np.array(kroB200)

In [ ]:
def get_start_nodes(matrix):
    first_start=rd.randint(0, 199)
    second_start=np.argmax(matrix[first_start])
    return first_start,second_start

In [ ]:
def create_random_cycles(instance):
    vertices = np.arange(0, len(instance))
    rd.shuffle(vertices)
    
    split_index = len(instance) // 2
    
    # return cycle1 and cycle2
    return vertices[:split_index], vertices[split_index:]

In [ ]:
#utworzenie dwóch list (cyklów) i dodanie wierzchołków startowych oraz utworzenie listy z pozostałymi wierzchołkami
def create_cycles(first_start, second_start):
    cycle1=[]
    cycle2=[]
    remaining = list(range(200))
    remaining.remove(first_start)
    remaining.remove(second_start)
    cycle1.append(first_start)
    cycle2.append(second_start)
    return cycle1, cycle2, remaining

In [ ]:
#algorytm zachłanny inspirowany metodą najbliższego sąsiada
def nearest_neighbour(matrix, cycle, remaining):
    min_distance = matrix[cycle[0]][remaining[0]]
    min_node = remaining[0]
    node_cycle = cycle[0]
    for node_to_add in remaining:
        for node_in_cycle in cycle:
            if matrix[node_to_add][node_in_cycle] < min_distance:
                min_distance = matrix[node_to_add][node_in_cycle]
                min_node = node_to_add
                node_cycle = node_in_cycle
    return min_node, node_cycle

In [ ]:
#algorytm zachłanny inspirowany metodą rozbudowy cyklu
def greedy_cycle(matrix, cycle, remaining):
    min_distance = 999999999
    index = -1
    min_node = cycle[0]
    for node_to_add in remaining:
        for i in range (len(cycle)):
            if i!=len(cycle)-1:
                tmp_distance = matrix[cycle[i]][node_to_add]+ matrix[cycle[i+1]][node_to_add] - matrix[cycle[i]][cycle[i+1]]
            else:
                tmp_distance = matrix[cycle[i]][node_to_add]+ matrix[cycle[0]][node_to_add] - matrix[cycle[i]][cycle[0]]
            if tmp_distance < min_distance:
                min_distance = tmp_distance
                min_node = node_to_add
                index = i
    return min_node, index

In [ ]:
#funkcja zwracająca listę wierzchołków, dla których najlepsze miejsce wstawienia to miejsce w cyklu o podanym indeksie
def nodes_with_best_given_index(matrix,cycle,remaining,given_index):
    result_nodes = []
    for node_to_add in remaining:
        best1 = 9999999999998
        best2 = 9999999999999
        index = -1
        for i in range (len(cycle)):
            if i!=len(cycle)-1:
                tmp_distance = matrix[cycle[i]][node_to_add]+ matrix[cycle[i+1]][node_to_add] - matrix[cycle[i]][cycle[i+1]]
            else:
                tmp_distance = matrix[cycle[i]][node_to_add]+ matrix[cycle[0]][node_to_add] - matrix[cycle[i]][cycle[0]]
            if tmp_distance < best2:
                if tmp_distance < best1:
                    best2 = best1
                    best1 = tmp_distance
                    index = i
                else:
                    best2 = tmp_distance
        if index == given_index:
            result_nodes.append([node_to_add,best1,best2])
    return result_nodes

In [ ]:
def calculate_regret_with_weight(list_nodes):
    list_nodes = np.array(list_nodes)
    difference = list_nodes[:,2]-1.2*(list_nodes[:,1])
    best_result_index = np.argmax(difference)
    return int(list_nodes[best_result_index][0])

In [ ]:
def greedy_regret_with_weight(matrix, cycle, remaining):
    best_node_to_add, best_index = greedy_cycle(matrix, cycle, remaining)
    nodes_on_the_same_position = nodes_with_best_given_index(matrix, cycle, remaining, best_index)
    return calculate_regret_with_weight(nodes_on_the_same_position), best_index

In [ ]:
def calculate_greedy_regret_with_weight(matrix, cycle1, cycle2, remaining):
    node_to_add, node_in_cycle = nearest_neighbour(matrix, cycle1, remaining)
    index = cycle1.index(node_in_cycle)
    cycle1.insert(index+1, node_to_add)
    remaining.remove(node_to_add)

    node_to_add, node_in_cycle = nearest_neighbour(matrix, cycle2, remaining)
    index = cycle2.index(node_in_cycle)
    cycle2.insert(index+1, node_to_add)
    remaining.remove(node_to_add)

    node_to_add, index = greedy_cycle(matrix, cycle1, remaining)
    cycle1.insert(index+1, node_to_add)
    remaining.remove(node_to_add)

    node_to_add, index = greedy_cycle(matrix, cycle2, remaining)
    cycle2.insert(index+1, node_to_add)
    remaining.remove(node_to_add)

    while (remaining):
        node_to_add, index = greedy_regret_with_weight(matrix, cycle1, remaining)
        index = int(index)
        cycle1.insert(index+1, node_to_add)
        remaining.remove(node_to_add)
        if not remaining:
             break
        node_to_add, index = greedy_regret_with_weight(matrix, cycle2, remaining)
        index = int(index)
        cycle2.insert(index+1, node_to_add)
        remaining.remove(node_to_add)
    return([distance_sum(cycle1,matrix)+distance_sum(cycle2,matrix), cycle1, cycle2])

In [ ]:
# change_type -> 1 dla wymiany wierzchołków w cyklu, 2 dla wymiany krawędzi w cyklu
def generate_moves(cycle1, cycle2, change_type):
    
    # ruchy międzytrasowe
    moves1 = [(0, (index1, index2)) for index1 in range(len(cycle1)) for index2 in range(len(cycle2))]
    
    # ruchy wewnątrztrasowe
    moves2 = []
    # wymiana wierzchołków w cyklu
    if change_type == 1:
        moves2 = [(1, (cycle_number, index1, index2)) for cycle_number, cycle in enumerate([cycle1, cycle2]) 
                  for index1 in range(len(cycle)) for index2 in range(len(cycle)) if index1 != index2]
    
    moves3 = []
    # wymiana krawędzi w cyklu
    if change_type == 2:
        for cycle_number, cycle in enumerate([cycle1, cycle2]):
            for index1 in range(len(cycle)):
                for index2 in range(len(cycle)):
                    if abs(index1 - index2) < 2:
                        continue
                        
                    if index1 == len(cycle) - 1:
                        index1_end = 0
                    else:
                        index1_end = index1 + 1
                        
                    if index2 == len(cycle) - 1:
                        index2_end = 0
                    else:
                        index2_end = index2 + 1
                        
                    move = (2, (cycle_number, index1, index1_end, index2, index2_end))
                    moves3.append(move)
                    
    return moves1 + moves2 + moves3

In [ ]:
def calculate_delta(cycle1, cycle2, matrix, move):
    # wymiana wierzchołków między cyklami
    if move[0] == 0:
        index1, index2 = move[1][0], move[1][1]
        
        node1, node2 = cycle1[index1], cycle2[index2]
        node1_before, node2_before = cycle1[index1-1], cycle2[index2-1]
        node1_after, node2_after = cycle1[(index1+1) % len(cycle1)], cycle2[(index2+1) % len(cycle2)]
        
        delta = matrix[node1_before][node2] + matrix[node2][node1_after] + \
                matrix[node2_before][node1] + matrix[node1][node2_after] - \
                matrix[node1_before][node1] - matrix[node1][node1_after] - \
                matrix[node2_before][node2] - matrix[node2][node2_after]
     
    # wymiana wierzchołków w cyklu
    elif move[0] == 1:
        cycle = cycle1 if not move[1][0] else cycle2
        
        index1, index2 = move[1][1], move[1][2]
        
        node1, node2 = cycle[index1], cycle[index2]
        node1_before, node2_before = cycle[index1-1], cycle[index2-1]
        node1_after, node2_after = cycle[(index1+1) % len(cycle)], cycle[(index2+1) % len(cycle)]
        
        old_distance = matrix[node1][node1_before] + matrix[node1][node1_after] + matrix[node2][node2_before] + matrix[node2][node2_after]
        new_distance = matrix[node1][node2_before] + matrix[node1][node2_after] + matrix[node2][node1_before] + matrix[node2][node1_after]
        
        delta = new_distance - old_distance
        
    # wymiana krawędzi w cyklu
    else:
        cycle = cycle1 if not move[1][0] else cycle2
        index1, index1_2, index2, index2_2 = move[1][1:]
        node1, node1_2, node2, node2_2 = cycle[index1], cycle[index1_2], cycle[index2], cycle[index2_2]
        
        delta = matrix[node1][node2] + matrix[node1_2][node2_2] - matrix[node1][node1_2] - matrix[node2][node2_2]
        
    return delta

In [ ]:
def apply_move(cycle1, cycle2, move):
    # ruch pomiędzy cyklami
    if move[0] == 0:
        temp = cycle1[move[1][0]]
        cycle1[move[1][0]] = cycle2[move[1][1]]
        cycle2[move[1][1]] = temp
        
    # ruch w cyklu -> wymiana wierzchołków
    elif move[0] == 1:
        cycle = cycle2 if move[1][0] else cycle1

        temp = cycle[move[1][1]]
        cycle[move[1][1]] = cycle[move[1][2]]
        cycle[move[1][2]] = temp
     
    # ruch w cyklu -> wymiana krawędzi
    elif move[0] == 2:
        cycle = cycle2 if move[1][0] else cycle1
        index1, index1_2, index2, index2_2 = move[1][1:]
        if index1_2 == 0:
            c1 = [cycle[index1_2]]
            c2 = cycle[index2_2:index1 + 1]
            c3 = cycle[index2:index1_2:-1]
        elif index2_2 == 0:
            c1 = cycle[:index1_2]
            c2 = cycle[index2:index1:-1]
            c3 = []
        else:
            c1 = cycle[:min(index1_2, index2_2)]
            c2 = cycle[max(index1_2, index2_2)-1 : min(index1_2, index2_2)-1 : -1]
            c3 = cycle[max(index1_2, index2_2):]
        
        if type(c1) is np.ndarray:
            c1 = c1.tolist()
        if type(c2) is np.ndarray:
            c2 = c2.tolist()
        if type(c3) is np.ndarray:
            c3 = c3.tolist()
            
        if not isinstance(c1, list):
            c1 = [c1]
        if not isinstance(c2, list):
            c2 = [c2]
        if not isinstance(c3, list):
            c3 = [c3]
        
        cycle = c1 + c2 + c3
        
        if not move[1][0]:
            cycle1 = cycle
        else:
            cycle2 = cycle
            
    return cycle1, cycle2

In [ ]:
def greedy_local_search(cycle1, cycle2, matrix, change_type):
    cycle1_copy = cycle1.copy()
    cycle2_copy = cycle2.copy()
    
    moves = generate_moves(cycle1_copy, cycle2_copy, change_type)
    
    while True:
        no_better_moves = True
        random_bias = rd.randrange(0, len(moves))
        random_direction = rd.choice([-1, 1])
        
        for i in range(len(moves)):
            move = moves[random_direction * ((i + random_bias) % len(moves))]
            delta = calculate_delta(cycle1_copy, cycle2_copy, matrix, move)
            if delta < 0:
                cycle1_copy, cycle2_copy = apply_move(cycle1_copy, cycle2_copy, move)
                no_better_moves = False
                break

        if no_better_moves:
            break
    return cycle1_copy, cycle2_copy, distance_sum_old(cycle1_copy,matrix)+distance_sum_old(cycle2_copy,matrix)

In [ ]:
def greedy_local_search_new(cycle1, cycle2, matrix, change_type):
    cycle1_copy = cycle1.copy()
    cycle2_copy = cycle2.copy()
    
    moves = generate_moves(cycle1_copy, cycle2_copy, change_type)
    
    while True:
        no_better_moves = True
        random_bias = rd.randrange(0, len(moves))
        random_direction = rd.choice([-1, 1])
        
        for i in range(len(moves)):
            move = moves[random_direction * ((i + random_bias) % len(moves))]
            delta = calculate_delta(cycle1_copy, cycle2_copy, matrix, move)
            if delta < 0:
                cycle1_copy, cycle2_copy = apply_move(cycle1_copy, cycle2_copy, move)
                no_better_moves = False
                break

        if no_better_moves:
            break
    return cycle1_copy, cycle2_copy, distance_sum_old(cycle1_copy,matrix)+distance_sum_old(cycle2_copy,matrix)

In [ ]:
def get_20_random_pairs(matrix):
    pairs = []
    for _ in range(20):
        cycle1, cycle2 = create_random_cycles(matrix)
        cycle1, cycle2, score = greedy_local_search(cycle1, cycle2, matrix, 2)
        scores = []
        while score in scores:
            cycle1, cycle2 = create_random_cycles(matrix)
            cycle1, cycle2, score = greedy_local_search(cycle1, cycle2, matrix, 2)
        scores.append(score)
        new_cycle1 = []
        new_cycle2 = []
        for i in range(len(cycle1)):
            new_cycle1.append((cycle1[i], cycle1[(i+1)%len(cycle1)]))
            new_cycle2.append((cycle2[i], cycle2[(i+1)%len(cycle2)]))
        distance = distance_sum(new_cycle1,matrix)+distance_sum(new_cycle2,matrix)
        pairs.append([new_cycle1, new_cycle2, distance])
    pairs = np.array(pairs)
    pairs = pairs[pairs[:, 2].argsort()]
    return pairs

In [ ]:
print(get_20_random_pairs(kroA200)[0])

In [ ]:
def get_2_random(pairs):
    first=rd.randint(0, len(pairs)-1)
    second=rd.randint(0, len(pairs)-1)
    while first==second:
        second=rd.randint(0, len(pairs)-1)
    return pairs[first], pairs[second]

In [ ]:
def get_common_edges(first, second):
    common_edges1 = []
    common_edges2 = []
    reverse_second1 = reverseTuple(second[0])
    reverse_second2 = reverseTuple(second[1])
    for edge in first[0]:
        if edge in second[0] or edge in reverse_second1 or edge in second[1] or edge in reverse_second2:
            common_edges1.append(edge)
    for edge in first[1]:
        if edge in second[0] or edge in reverse_second1 or edge in second[1] or edge in reverse_second2:
            common_edges2.append(edge)
    return common_edges1, common_edges2

In [ ]:
first, second = get_2_random(get_20_random_pairs(kroA200))
cycle1, cycle2 = get_common_edges(first, second)

In [ ]:
def find_end(p_start, p_cycle, p_left_cycle):
    if p_start in p_left_cycle:
        for i, j in p_cycle:
            if i == p_start: return j 
    return -100

In [ ]:
def find_cycle_from_edge(p_index, p_cycle, p_edge, p_visited, p_left_cycle):
    p_visited[p_index] = True
    
    s, e = p_edge
    new_s = e
    new_e = find_end(new_s, p_cycle, p_left_cycle)

    if new_e == -100:
        return False
    
    new_edge = new_s, new_e
    new_index = p_cycle.index(new_edge)
    
    if p_visited[new_index]:
        return True
        
    else:
        return find_cycle_from_edge(new_index, p_cycle, new_edge, p_visited, p_left_cycle)

In [ ]:
def find_cycle(p_cycle):
    left_cycle = [i for i, j in p_cycle]
    
    visited_edges = [False] * len(p_cycle) 
    
    for index, edge in enumerate(p_cycle):
        if visited_edges[index] == False:
            res = find_cycle_from_edge(index, p_cycle, edge, visited_edges, left_cycle)
            if res:
                return True

    return False

In [ ]:
def get_nodes_to_add_after(cycle):
    left_new_cycle = [i for i, j in cycle]
    right_new_cycle = [j for i, j in cycle]
    nodes = []
    for i in right_new_cycle:
        if i not in left_new_cycle:
            nodes.append(i)
    return nodes

In [ ]:
def get_nodes_to_add_before(cycle):
    left_new_cycle = [i for i, j in cycle]
    right_new_cycle = [j for i, j in cycle]
    nodes = []
    for i in left_new_cycle:
        if i not in right_new_cycle:
            nodes.append(i)
    return nodes

In [ ]:
def get_nodes_not_in_cycles(cycle1, cycle2):
    new_cycle_vertices = list(set(cycle1+cycle2))
    result = []
    all_nodes = list(range(200))
    for i in all_nodes:
        if i not in new_cycle_vertices:
            result.append(i)
    return result

In [ ]:
def generate_all_moves(cycle1, cycle2):
    cycle1_before = get_nodes_to_add_before(cycle1)
    cycle1_after = get_nodes_to_add_after(cycle1)
    cycle2_before = get_nodes_to_add_before(cycle2)
    cycle2_after = get_nodes_to_add_after(cycle2)
    nodes_not_in_cycles = get_nodes_not_in_cycles(cycle1, cycle2)
    all_moves = []
    for node in nodes_not_in_cycles:
        for x in cycle1_before:
            all_moves.append([(node,x), 1]) # 1 - cykl1
        for x in cycle2_before:
            all_moves.append([(node,x), 2]) # 2 - cykl2
        for x in cycle1_after:
            all_moves.append([(x,node), 1]) # 1 - cykl1
        for x in cycle2_after:
            all_moves.append([(x,node), 2]) # 2 - cykl2
    for x in cycle1_before:
        for y in cycle1_after:
            all_moves.append([(y,x),1])
    for x in cycle2_before:
        for y in cycle2_after:
            all_moves.append([(y,x),2])
    return all_moves

In [ ]:
def check_if_move_possible(move,node_cycle,another_cycle):
    
    
    another_cycle_vertices = [i for i, j in another_cycle] + [j for i, j in another_cycle]
    another_cycle_vertices = list(set(another_cycle_vertices))
    node_cycle_vertices = [i for i, j in node_cycle] + [j for i, j in node_cycle]
    node_cycle_vertices = list(set(node_cycle_vertices))
    node_cycle_after = get_nodes_to_add_after(node_cycle)
    node_cycle_before = get_nodes_to_add_before(node_cycle) 
    test = list(range(200))
#     print('----------------------------------------------------')
#     print(len(node_cycle))
#     print(len(node_cycle_vertices))
#     print(node_cycle)
#     print(len(another_cycle))
#     print(len(another_cycle_vertices))
#     print(another_cycle)
#     print("CHECK")
#     print(move)
#     print("end")
#     print(get_nodes_to_add_after(node_cycle))
#     print('----------------------------------------------------')
#     print("begin")
#     print(get_nodes_to_add_before(node_cycle))
#     print('----------------------------------------------------')
#     print("Brakujące:")
#     for x in test:
#         if x not in another_cycle_vertices and x not in node_cycle_vertices and len(node_cycle_vertices)<100:
#             print(x)
#     print('----------------------------------------------------')
    if len(node_cycle_vertices) == 100 and move[1] not in node_cycle_before:
        return False
    if move[1] in another_cycle_vertices:
        return False
#     print("CHECK2")
    if move[1] not in node_cycle_vertices and move[1] not in another_cycle_vertices and len(node_cycle_vertices)<100:
        return True
#     print("CHECK3")
    temp_cycle = node_cycle.copy()
    temp_cycle.append(move)
    if find_cycle(temp_cycle) and len(node_cycle_vertices)<100: #and len(temp_cycle)<100:
    #if find_start_edge(node_cycle,move[0])==move[1] and len(node_cycle_vertices)<100:
        return False
#     print(find_start_edge(node_cycle,move[0]))
#     print(move[1])
#     print("CHECK4")
       
#     print('----------------------------------------------------')
#     print(node_cycle_after)
#     print(node_cycle_before)
#     print('----------------------------------------------------')
    if move[1] in node_cycle_after or move[1] in node_cycle_before:
        return True
    node_cycle_vertices = [i for i, j in node_cycle] + [j for i, j in node_cycle]
    node_cycle_vertices = list(set(node_cycle_vertices))
    if move[1] in node_cycle_vertices:
        return False
    return False

In [ ]:
# def get_best_node(node,node_cycle,another_cycle,matrix):
#     distances = matrix[node].copy()
#     distances.sort()
#     distances.remove(0)
#     best_distance = distances[0]
#     node_to_add = matrix[node].index(best_distance)
# #     print(node_cycle)
# #     print(another_cycle)
# #     print(node)
# #     print('----------------------------------------------------')
#     while(check_if_move_possible((node,node_to_add),node_cycle,another_cycle)==False):
# #         print(node_to_add)
#         distances.remove(best_distance)
#         if len(distances)==0:
#             print(node)
#             print(node_cycle)
#             print(another_cycle)
#             print(get_nodes_to_add_after(node_cycle))
#             print(get_nodes_to_add_before(node_cycle))
#             print(get_nodes_to_add_after(another_cycle))
#             print(get_nodes_to_add_before(another_cycle))
#         best_distance = distances[0]
#         node_to_add = matrix[node].index(best_distance)
#     return node_to_add

In [ ]:
def get_best_node(node,node_cycle,another_cycle,matrix):
    distances = []
    index = 0
    row = matrix[node].copy()
    for distance in row:
        if distance !=0 :
            distances.append([distance,index])
        index+=1
    distances = np.array(distances)
    distances = distances[distances[:, 0].argsort()]
    distances = list(distances)
    index = 0
    node_to_add = int(distances[0][1])
#     if len(distances)==199:
#         print(distances)
#     print(node_cycle)
#     print(another_cycle)
#     print(node)
#     print('----------------------------------------------------')
    while(check_if_move_possible((node,node_to_add),node_cycle,another_cycle)==False):
#         print(node_to_add)
        index+=1

#         if len(distances)==0:
#             print(node)
#             print(node_cycle)
#             print(another_cycle)
#             print(get_nodes_to_add_after(node_cycle))
#             print(get_nodes_to_add_before(node_cycle))
#             print(get_nodes_to_add_after(another_cycle))
#             print(get_nodes_to_add_before(another_cycle))
        node_to_add = int(distances[index][1])
    return node_to_add

In [ ]:
# def find_start(p_end, p_cycle, p_left_cycle):
#     if p_start in p_left_cycle:
#         for i, j in p_cycle:
#             if i == p_end: return j 
#     return -100

In [ ]:
def reverse_edges(cycle,node):
    left_cycle = [i for i, j in cycle]
    right_cycle = [j for i, j in cycle]
    while (node in right_cycle):
        ind = right_cycle.index(node)
        cycle.remove((left_cycle[ind],right_cycle[ind]))
        cycle.append((right_cycle[ind],left_cycle[ind]))
        node = left_cycle[ind]
    return cycle

In [ ]:
def find_start_of_edge(cycle, node):
    left_cycle = [i for i, j in cycle]
    right_cycle = [j for i, j in cycle]
    for i in range(len(right_cycle)):
        if right_cycle[i]==node:
            return left_cycle[i]

In [ ]:
def find_start_edge(cycle, node):
    left_cycle = [i for i, j in cycle]
    right_cycle = [j for i, j in cycle]
    new_node = find_start_of_edge(cycle,node)
    while (new_node in left_cycle):
        ind = left_cycle.index(new_node)
        new_node = right_cycle[ind]
    result = find_start_of_edge(cycle,new_node)
    return result

In [ ]:
def repair(cycle1,cycle2,matrix):
    nodes1 = [i for i, j in cycle1] + [j for i, j in cycle1]
    nodes1 = list(set(nodes1))
    nodes2 = [i for i, j in cycle2] + [j for i, j in cycle2]
    nodes2 = list(set(nodes2))
    while len(cycle1)<100 or len(cycle2)<100:
#         start_node_cycle1 = None
#         start_node_cycle2 = None
        
        if len(cycle1)<100:
#             print("111111111111111111111111111111111111")
            while len(cycle1)<=len(cycle2):
                left_cycle1 = [i for i, j in cycle1]
                cycle1_after = get_nodes_to_add_after(cycle1)
#                 for edge in cycle1:
#                     if (find_end(edge[1], cycle1, left_cycle1) == -100):
#                         start_node_cycle1 = edge[1]
#                         break
#                 print('------STARTNODE1------')
#                 print(start_node_cycle1)
#                 print('----------------------')
                node_to_add_cycle1 = get_best_node(cycle1_after[0],cycle1,cycle2,matrix)
                if node_to_add_cycle1 in cycle1_after:
                    cycle1 = reverse_edges(cycle1,node_to_add_cycle1)
#                 else:
#                     cycle1.append((cycle1_after[0], node_to_add_cycle1))
                cycle1.append((cycle1_after[0], node_to_add_cycle1))
                nodes1 = [i for i, j in cycle1] + [j for i, j in cycle1]
                nodes1 = list(set(nodes1))

        if len(cycle2)<100:
#             print("2222222222222222222222222222222222222222222")
            while len(cycle1)>len(cycle2):
                left_cycle2 = [i for i, j in cycle2]
                cycle2_after = get_nodes_to_add_after(cycle2)
                cycle2_before = get_nodes_to_add_before(cycle2)
#                 print(cycle2_after)
#                 print(cycle2_before)
#                 print(cycle2)
#                 print(len(cycle2))
#                 for edge in cycle2:
#                     if (find_end(edge[1], cycle2, left_cycle2) == -100):
#                         start_node_cycle2 = edge[1]
#                         break
#                 print('------STARTNODE2------')
#                 print(start_node_cycle2)
#                 print('----------------------')
                node_to_add_cycle2 = get_best_node(cycle2_after[0],cycle2,cycle1,matrix)
                cycle2_after = get_nodes_to_add_after(cycle2)
                if node_to_add_cycle2 in cycle2_after:
                    cycle2 = reverse_edges(cycle2,node_to_add_cycle2)
#                 else:
#                     cycle2.append((cycle2_after[0], node_to_add_cycle2))
                cycle2.append((cycle2_after[0], node_to_add_cycle2))
                nodes2 = [i for i, j in cycle2] + [j for i, j in cycle2]
                nodes2 = list(set(nodes2))
    return cycle1, cycle2

In [ ]:
def measure_hae_no_ls(matrix, time_limit):
    elite = get_20_random_pairs(matrix)
    start=time.time()
    while time.time()-start<time_limit:
        parent1, parent2 = get_2_random(elite)
        child1, child2 = get_common_edges(parent1, parent2)
#         print("common")
#         print(child1)
#         print(child2)
#         print("----------------")
        child1, child2 = repair(child1, child2, matrix)
#         print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        score = distance_sum(child1,matrix)+distance_sum(child2,matrix)
        elite = list(elite)
        elite.append([child1, child2, score])
        elite = np.array(elite)
        elite = elite[elite[:, 2].argsort()]
        elite = elite[:20]
    return elite[0]

In [ ]:
def measure_hae_with_ls(matrix, time_limit):
    elite = get_20_random_pairs(matrix)
    start=time.time()
    while time.time()-start<time_limit:
        parent1, parent2 = get_2_random(elite)
        child1, child2 = get_common_edges(parent1, parent2)
#         print("common")
#         print(child1)
#         print(child2)
#         print("----------------")
        child1, child2 = repair(child1, child2, matrix)
        print(child1)
        print("1--------------------------------")
        child1 = convert_2_nodes(child1)
        child2 = convert_2_nodes(child2)
        print(child1)
        print("2--------------------------------")
        child1, child2, _ = greedy_local_search(child1, child2, matrix, 2)
#         print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        child1 = convert_2_edges(child1)
        child2 = convert_2_edges(child2)
        print(child1)
        print("3--------------------------------")
        score = distance_sum(child1,matrix)+distance_sum(child2,matrix)
        elite = list(elite)
        elite.append([child1, child2, score])
        elite = np.array(elite)
        elite = elite[elite[:, 2].argsort()]
        elite = elite[:20]
    return elite[0]

In [ ]:
def convert_2_nodes(cycle):
    print(cycle)
    cycle_copy = cycle.copy()
    left_cycle = [i for i, j in cycle]
    result = []
    ind = 0
    pair = cycle_copy[ind]
    result.append(pair[0])
    result.append(pair[1])
    del cycle_copy[ind]
    del left_cycle[ind]
    while cycle_copy:
        ind = left_cycle.index(pair[1])
        print(cycle_copy)
        print(pair)
        print(ind)
        pair = cycle_copy[ind]
        result.append(pair[1])
#             print(cycle_copy)
#             print(cycle)
#             print(pair)
        del cycle_copy[ind]
        del left_cycle[ind]
    return result[:-1]

In [ ]:
def convert_2_edges(cycle):
    new_cycle = []
    for i in range(len(cycle)):
        new_cycle.append((cycle[i], cycle[(i+1)%len(cycle)]))
    return new_cycle

In [ ]:
dummy = [(0,1), (2,0), (1,2)]
print(convert_2_nodes(dummy))

In [ ]:
result = measure_hae_no_ls(kroB200,80)
print(result)

In [ ]:
cycle1 = result[0]
cycle2 = result[1]
cycle1 = convert_2_nodes(cycle1)
cycle2 = convert_2_nodes(cycle2)

In [ ]:
print(cycle1)

In [ ]:
plt.rc('figure', dpi=110, figsize=(9, 5))
sns.set_style('darkgrid')

cycle1.append(cycle1[0])
cycle2.append(cycle2[0])

coordinate_x_cycle1 = []
coordinate_y_cycle1 = []
coordinate_x_cycle2 = []
coordinate_y_cycle2 = []
for node in cycle1:
    coordinate_x_cycle1.append(instance2.loc[node]['x'])
    coordinate_y_cycle1.append(instance2.loc[node]['y'])
for node in cycle2:
    coordinate_x_cycle2.append(instance2.loc[node]['x'])
    coordinate_y_cycle2.append(instance2.loc[node]['y'])

# plotting the line 1 points
plt.plot(coordinate_x_cycle1, coordinate_y_cycle1, '-bo',  c='blue', mfc='k', mec='k', label="cycle1")
plt.plot(coordinate_x_cycle2, coordinate_y_cycle2, '-bo',  c='red', mfc='k', mec='k', label="cycle2")
# naming the x axis
plt.xlabel('x - axis')
# naming the y axis
plt.ylabel('y - axis')
# giving a title to my graph

# show a legend on the plot
plt.legend()

# function to show the plot
plt.show()